In [122]:
import sys
sys.path.append('/usr/lib/python3/dist-packages')

import os
import pandas as pd
import numpy as np
import requests
import json
import time
from IPython.display import clear_output
os.environ["DJANGO_SETTINGS_MODULE"] = "mysite.settings"
import django
from django.db.models import Sum
django.setup()
from tqdm import tqdm
from main_map.models import Organization, House, Metro, MetroStation, PolygonModel, RawLine, LineModel


In [ ]:
from pyproj import Proj, transform

PROJ3857  = Proj("+init=EPSG:3857")
PROJ4326 = Proj("+init=EPSG:4326")

In [ ]:
import itertools

In [123]:
SERVER_URL = 'http://178.62.250.186'
#SERVER_URL = 'http://127.0.0.1'
SERVER_URL_MSK_DRIVING = SERVER_URL + ':5000/table/v1/driving/'
SERVER_URL_MSK_FOOT = SERVER_URL + ':5001/table/v1/foot/'
SERVER_URL_PETER_DRIVING = SERVER_URL + ':5002/table/v1/driving/'
SERVER_URL_PETER_FOOT = SERVER_URL + ':5003/table/v1/foot/'

def get_time_3000(lon, lat, coords, root_url):
    page_url = root_url + '{},{};'.format(lon, lat)
    times = []
    for coord in coords:
        page_url = page_url + '{},{};'.format(coord[0], coord[1])
    page_url = page_url[:-1] + '?sources=0'
    page = requests.get(page_url)
    data = page.json()
    if 'durations' in data:
        times.extend(data['durations'][0][1:])
    else:
        print('no key name duration in data from osrm')
        times.extend([0]*len(coords))
    return times
def get_time(lon, lat, coords, mode):
    try:
        # Проверка на Питер
        if lat > 58 and lat > 58:
            root_url = {
                'driving': SERVER_URL_PETER_DRIVING,
                'foot': SERVER_URL_PETER_FOOT,
                }[mode]
        else:
            root_url = {
                'driving': SERVER_URL_MSK_DRIVING,
                'foot': SERVER_URL_MSK_FOOT,
                }[mode]
    except KeyError as e:
        raise ValueError('Undefined mode: {}'.format())
    times = []
    if len(coords) < 3000:
        times.extend(get_time_3000(lon,lat, coords, root_url))
    else:
        for coord_chunk in tqdm(np.array_split(coords, len(coords) / 3000)):
            times.extend(get_time_3000(lon, lat, coord_chunk, root_url))

    # Заменить все None на нули
    times = [0 if i is None else i for i in times]
    return times
def get_coord_time(x, y, step, rmax, mode):
    coords = get_coords(x, y, step, rmax)
    times = get_time(lon, lat, coords, mode)
    return coords, times
def get_coords(x, y, step, rmax):
    coords = itertools.product(np.arange(x - rmax, x + rmax, step),np.arange(y - rmax, y + rmax, step))
    coords = list(coords)
    #print(len(list(coords)))
    #print(list(coords)[0])
    coords = list(map(lambda coord: transform(PROJ3857, PROJ4326, coord[0], coord[1]), coords))
    #print(coords)
    return coords

In [126]:
lat, lon = 55.695314, 37.625357
x, y = transform(PROJ4326, PROJ3857, lon, lat)

In [127]:
transform(PROJ3857, PROJ4326,x, y)

coords, times = get_coord_time(x, y, 50, 5000, mode='driving')


(37.62535699999999, 55.695313999999975)